# Laboratoire 3 : Machines à vecteur de support et réseaux neuronaux
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | Alexandre Laroche - LARA12078907<br>Marc-Antoine Charland - CHAM16059609<br>Jonathan Croteau-Dicaire - CROJ10109402    |
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Été 2019                                            |
| Groupe                | 02                                                      |
| Numéro du laboratoire | TP-03                                                   |
| Professeur            | Prof. Alessandro L. Koarich                             |
| Chargé de laboratoire | Pierre-Luc Delisle                                                     |
| Date                  | 10 juillet 2019 (23h55)                                                    |

In [2]:
from collections import OrderedDict
from IPython.display import display, Markdown
import json
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models

import src.constants as constants

# Introduction

<br>
<div style="text-align: justify">
Bien que les réseaux de neurones artificiels multicouches (MLP) sont reconnus comme des algorithmes clés très performants en apprentissage machine, ceux-ci ne sont pas exempts de défauts. Plus particulièrement, les MLP n’accordent pas d’importance au choix des frontières de décision. Dans les faits, les MLP tracent ces frontières de façon à séparer les observations de chaque classe. Seulement, les MLP ne s’attaquent pas à la question d’où placer ces frontières entre les observations des classes. De ce fait, dans le cas d’un problème de classification binaire linéaire, un MLP peut être porté à tracer un hyperplan deux fois plus près des observations de la première classe que de ceux de la seconde classe. Heureusement, les machines à vecteurs de support (SVM) apportent une solution à ce problème. À cet égard, l’objectif du laboratoire présent est de comparer la performance des MLP et des SVM pour la tâche de classification automatique des galaxies en spirale et smooth. Par ailleurs, comme bonus ces deux modèles peuvent être comparés à un réseau de neurones convolutif (CNN). Parallèlement, ce laboratoire a aussi pour objectif d’étudier l’impact des hyperparamètres et de la structure des réseaux neuronaux sur la performance de ces modèles. Aussi, ce laboratoire va permettre aux étudiants de se familiariser avec ces algorithmes et les outils en apprentissage machine. Ainsi donc, ce rapport commence par expliquer le choix de la méthode de validation utilisée lors des expérimentations. Par la suite, la méthode de normalisation utilisée sera présentée. Puis, la conception et l’implémentation du MLP et du CNN seront énoncées. Cela donnera lieu à une courte étude sur l’état de surapprentissage de ces deux modèles et l’identification du nombre d’epochs optimal. D'ailleurs, une étude portera aussi sur l’impact des hyperparamètres sur la performance du MLP et du CNN. Une étude similaire aura pour sujet le SVM. Cependant, celle-ci sera présentée après une explication de la méthode utilisée pour trouver le meilleur modèle SVM. Ensuite, l’impact de la taille de l’ensemble de données sur la performance de chacun des modèles sera analysé. Partant de ces études et analyses, une recommandation sera émise pour le meilleur type de classifieur pour l’ensemble de données des galaxies. Enfin, une discussion portera sur les pistes d’amélioration de ces classifieurs.
</div>

# Question 1
<br>
<div style="text-align: justify">
La méthode de validation utilisée dans ce rapport est le Holdout. De ce fait, la répartition entre les sous-ensembles d'entraînement et de validation est de 80/20. Cette méthode est choisie, car elle est plus rapide que les méthodes qui divisent l’ensemble de données en plus de sous ensembles. À vrai dire, pour ce laboratoire, le temps est un enjeux important.
<br><br>
En effet, les réseaux de neurones artificiels (ANN) et les machines à vecteurs de supports (SVM) sont reconnues pour avoir un temps d'entraînement très variable. C’est-à-dire que le temps d'entraînement de ces modèles complexes ne dépend pas seulement de la tail de l’ensemble de données, mais aussi des hyperparamètres, de leur architecture, et de la complexité du problème. Notamment, les ANN ont la capacité d’être ajusté aux données d’apprentissage. Pour ce faire, les poids du réseau sont itérativement mit-à-jours afin de réduire l’erreur. Chacune de ces itérations dénote une époque. Ainsi, pour un même problème, certains ANN nécessitent un plus grands nombre d’époques pour atteindre le même niveau d’apprentissage que des ANN plus performants. Pour sa part, le temps d’apprentissage des SVM dépend entre autre les la fonction noyau choisie. Une fonction plus complexe et plus coûteuse en temps tel que RBF est parfois nécessaire pour atteindre des performances acceptables. Cependant, la conception des modèles d’apprentissage n’est pas le seul enjeux de temps pour ce laboratoire.
<br><br>
À cet égard, les primitives utilisées pour l'entraînement doivent êtres choisies. En réalité, entraîner un SVM et un ANN sur les 74 primitives disponibles garantit un temps d'entraînement élevé. De plus, ce rapport offre en bonus d’implémenter un réseau convolutif (CNN). Cette autre architecture de réseau de neurones artificiel est aussi reconnue pour être parfois long à entraîner. Dans les faits, celui-ci prend en entrée comme primitive les pixels des images. Ainsi, une image d’une taille de 140x140 pixels représente 140 * 140 = 19 600 primitives. 
<br><br>
Compte tenue de ce qui précède, pour obtenir des résultats intéressant au cours de ce laboratoire, de très nombreuses configurations d’algorithmes doivent êtres testées. De plus, le choix à été fait d’implémenter le CNN en bonus. Ce bonus vas demander une quantité de temps considérable. Ainsi, le temps est une contrainte majeur pour ce travail. De ce fait, c’est logique de favoriser une économie du temps au détriment de la qualité des résultats. C’est exactement ce que la méthode de validation Holdout offre. Contrairement, aux méthodes qui considèrent la moyenne des résultats de plusieurs exécutions tel que K-folds, la méthode Holdout ne nécessite pas d’exécuter K entraînements à chaque fois qu’un hyperparamètre est changé. Seulement, cette méthode donne des résultats qui sont moins représentatif des résultats espérés que ceux de ces autres méthodes.
</div>

# Question 2
<br>
<div style="text-align: justify">
Dans un premier temps, il est important de souligner qu’il n’existe pas de méthode unique de normalisation (mise à l’échelle) pour l’ensemble des modèles d’apprentissages. En effet, certaines méthodes améliorent les résultats de certains modèles et certaines méthodes n’ont tout simplement pas d’impacts significatifs sur certains modèles. Par exemple, il a été observé dans les laboratoires précédents que l’arbre de décisions est plus ou moins affecté par la mise à l’échelle du jeu de données. Bref, l’impact des méthodes de mise à l’échelle sur les classificateurs n’ont pas toujours le même effet car, chaque classificateur fonctionne différemment. 
<br><br>
Dans le cas du laboratoire présent, la méthode StandardScaler, MinMaxScaler et PowerTransformer de sklearn ont été approchées pour mettre à l’échelle le jeu de données du MLP et du SVM. 
<br><br>
Pour le MLP, plusieurs tests ont été effectués pour identifier la meilleure méthode de mise à l’échelle. Les méthodes telles que StandardScaler, MinMaxScaler et PowerTransformer ont été utilisées. Ces méthodes ont été choisies basées sur celle que l’on voyait le plus souvent dans nos recherches sur le sujet. Après exécution du baseline avec chacune de ces méthodes il a été remarqué que StandardScaler offrait la meilleure accuracy suivi de près par PowerTransformer. En échange MinMaxScaler donnait un accuracy considérablement plus faible.
</div>

![standardscaler](logs/NN/nn_standardscaler.png)

<div style="text-align: justify">
La ligne orange du graphique ci-dessous représente le baseline utilisé avec le prétraitement de StandardScaler et la ligne en bleu représente le même baseline exécuté avec MinMaxScaler.
<br><br>
Pour ce qui est du modèle SVM, la méthode StandardScaler de sklearn a été utilisée pour normaliser le jeu de données. La fonction en question normalise les données en supprimant la moyenne et en effectuant une mise à l’échelle à la variance d’unité. Autre que sa grande rapidité d’exécution, la raison de son utilisation est simple. Par exemple, le SVM avec noyau RBF suppose que toutes les données sont centrées autour de 0 et présentent une variance dans le même ordre. Si une caractéristique a une variance supérieure de plusieurs ordres de grandeur, elle peut rendre l’estimateur incapable d’apprendre correctement.
<br><br>
Pour justifier encore plus l’utilisation de la méthode StandardScaler de sklearn avec RBF, un court exemple est utilisé pour appuyer cette prise de décision. Dans un premier temps, on simplifie la fonction du RBF pour faciliter la démonstration et on spécifie trois vecteurs pour cet exemple.
</div>

K( u , v ) = exp( - || u - v || ^ 2 )

X1 = [ 1000 , 1 , 2 ]

X2 = [ 900 , 1 , 2 ]

X3 = [ 1050 , -10 , 20 ]

<div style="text-align: justify">
<br>
Deuxièmement, on insère les vecteurs dans la fonction pour voir ce qui se passe. On suppose que le vecteur X1 est plus similaire à X3 que X2.
</div>

K( X1 , X2 ) = exp( -10000 )

K( X1 , X3 ) = exp( -2905 )

<div style="text-align: justify">
<br>
Troisièmement, on analyse les différences entre les vecteurs X1 et X2 ainsi que X1 et X3.
</div>

X2 = [ 0.1 , 0 , 0 ]

X3 = [ 0.05 , -10 , 10 ]

<div style="text-align: justify">
<br>
Les résultats obtenus permettent de dire que sans la mise à l’échelle, le vecteur X1 est plus similaire à X3 qu’à X2 même si les différences relatives par caractéristique entre X1 et X3 sont beaucoup plus grandes que celles de X1 et X2. En d’autres mots, si la mise à l’échelle de la méthode StandardScaler de sklearn n’est pas appliquée sur toutes les valeurs à des plages comparables, les valeurs dont la plage est plus grande dominent complètement dans le calcul de la matrice du noyau. 
</div>

# Question 3

### MLP
<br>
<div style="text-align: justify">
Le modèle d’apprentissage est basé sur le modèle de base fourni dans l’énoncé fournie avec le laboratoire. En effet, le modèle de base est composé de 4 couches soit celle d’entrée, deux de 100 perceptrons et une dernière de 2 perceptrons pour la sortie. Toutes les couches excepté la dernière utilisent la fonction d’activation ReLU. Celle-ci est moins susceptible au problème du vanishing gradient. Ce problème survient avec des fonctions d’activation telle que sigmoid. Ce problème peut être décrit comme le fait que lors de la propagation arrière, le gradient devient de plus en plus petit. En conséquence, cette réduction du gradient peut empêcher le réseau d’apprendre. Pour sa part, la dernière couche utilise la fonction d’activation Softmax.
<br><br>
L’implémentation a été faite de façon légèrement plus flexible que ce qui avait été demandé pour le modèle de base. En effet, étant donné le besoin de changer le nombre de couches et de changer certain hyperparamètre.
</div>

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(layers[0], input_shape=(x.shape[1],), activation='relu'))
for index in range(1, len(layers)-1):
    model.add(keras.layers.Dense(layers[index], activation='relu'))
model.add(keras.layers.Dense(layers[len(layers)-1], activation='softmax'))

<div style="text-align: justify">
Comme on peut voir dans cet extrait de la méthode d’entraînement, on utilise un tableau d’entier nommé layers pour créer le bon nombre de couches avec. L’appel de la fonction se fait comme ci-dessous :
</div>

In [ ]:
results.append(executeNN(x, y, "NN/BaseLine",0.005,60, [100,100,2]))

<div style="text-align: justify">
Results est simplement une variable qui enregistre des tableaux de résultats pour l’affichage par la suite. executeNN est la méthode qui contient l’exécution du réseau de neurones. On peut ici voir dans l’ordre : le tableau de X utiliser, le tableau de Y utilisé, le nom qui sera donné a cette exécution, le learning rate, le nombre epoch a faire et finalement un array d’entier représentant chacune des couches.
<br><br>
La fonction de coût utilisée est categorical cross-entropy. Selon Michael Nielsen « cross-entropy is a widely-used cost function » (Nielsen, 2015). Celle-ci est la fonction la plus recommandée pour les problèmes de classification. Cela est dû au fait que celle-ci est équivalente à la fonction de vraisemblance logarithmique (Nielsen, 2015). Afin de maximiser l’estimation de la vraisemblance, on peut minimiser le négatif du logarithme de la vraisemblance. De ce fait, c’est logique d’utiliser la cross-entropy comme fonction de coût pour un problème de classification. Notez que nous avons utilisé la version catégorique et non pas la version binaire. Cela est une erreur d'inattention. Heureusement, la version catégorique peut aussi être utilisée pour les problèmes binaires.
<br><br>
Parallèlement, la fonction Mean Squared Error est recommandée pour les problèmes de régressions. Ceci explique pourquoi nous n’avons pas choisi cette fonction.
</div>

### CNN
<br>
<div style="text-align: justify">
Les réseaux neuronaux convolutif (CNN) ajoutent une multitude de caractéristiques à la définition standard des réseaux de neurones artificiels. Ces caractéristiques expliquent pourquoi les CNN sont particulièrement utilisées pour la classification d’images. Contrairement au MLP, les CNN sont invariantes à l’espace. De plus, ceux-ci ont la capacité de traiter des données multidimensionnelles. Parallèlement, les CNN peuvent effectuer une extraction automatique des primitives. Ces prouesses sont le résultat de l’utilisation des convolutions. Dans les faits, un CNN est un perceptron à une couche qui suit une succession de couches de convolution et de pooling tel que présenté dans le code du CNN baseline ci-dessous.    
</div>

<br>
<div style="text-align: justify; text-decoration: underline;">Code du modèle baseline du CNN</div>

In [ ]:
model = Sequential()
    
model.add(Conv2D(filters=3, kernel_size=(3, 3), padding='same', 
                 input_shape=(140, 140, 3)))
model.add(Activation('relu'))
model.add(Conv2D(filters=3, kernel_size=(3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=6, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(filters=6, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=12, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(filters=12, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=24, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(filters=24, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(
    optimizer=RMSprop(), 
    loss='categorical_crossentropy'
)

<div style="text-align: justify">
Les couches de convolution extraient des features maps des images. En termes simples, les convolutions sont une opération mathématique pour combiner deux matrices. La première est une section des pixels de l’image. La deuxième est un noyau contenant des poids. Dans le modèle présenté ci-dessus, les noyaux ont une dimension de 3x3 pixels. Ainsi, une convolution revient à faire glisser ce noyau sur tous les pixels de l’image. Ce noyau commence dans le coin haut gauche de l’image et glisse vers la droite un pixel à la fois. Une fois que le noyau atteint le côté droit de l’image, il recommence à gauche un pixel sous la position du dernier glissement. La taille de la features map produite par une convolution dépend de la taille de l’image, des dimensions du noyau et du nombre de glissement nécessaire. En fonction de ces éléments, la features map peut parfois être de dimension inférieure à celle de l’image. Ce type de features map est dite valid. Ainsi, une couche de convolution valide permet de réduire la dimensionnalité des primitives. Afin d’obtenir une features map de même dimension que l’image; dite same, une marge de pixels vides peut être ajoutée autour de l’image. Dans le CNN présenté, chaque couche de convolution valid est précédée d’une couche same. Cela permet d’utiliser un plus grand nombre de couches de convolutions avant que la features map ne puisse plus être réduite. Ce plus grand nombre de couches de convolution permet d’extraire plus de primitives de l’image.
<br><br>
Par la suite, chaque couche de convolution est suivie d’une couches d’activation. Ces couches d’activations utilisent la fonction ReLU. Par ailleurs, chaque groupe de couches de convolution same et valid sont suivit d’une couche de max pooling. Ce type de couche permet une fois de plus de réduire la dimensionnalité des primitives extraites. Le fonctionnement de cette couche est similaire à celui d’une couche de convolution. C’est-à-dire qu’une fenêtre glisse sur la matrice d’entrée. Cependant, les couches de pooling ont en entrée le features maps des couches de convolution précédentes. Aussi, les couches de max pooling retiennent seulement les valeurs maximales dans une fenêtre.
<br><br>
Enfin, le CNN se termine en transformant la sortie de la dernière couche de pooling en un vecteur à une seule dimension. Ce vecteur est ensuite passé à une couche de perceptrons qui à la même fonction que la couche de sortie dans un MLP.
</div>

# Question 4

### MLP
<br>

![val_loss_200epochs](logs/NN/nn_val_loss_200epochs.png)

<div style="text-align: justify">
Le MLP présenté ci-dessus a été exécuté sur 200 epochs. On peut voire sur cette image que le loss descend entre les epochs [1, 5] puis recommence à monter après l’epoch 6 donc de [7, 200]. On peut donc en déduire que le MLP est en état de sous apprentissage avant l’epoch 6 et qu'après l’époque 6 il est en surapprentissage. Donc le nombre d’epochs optimal est de 6 dans le cas du MLP.
</div>

### CNN
<br>
<div style="text-align: justify">
Concernant le CNN, la figure  4.cnn.1 présente la courbe du loss lors de la validation du CNN baseline au cours de 100 epochs. Cette figure montre que le loss décroît entre les epochs [1, 8]. Par la suite, le loss tend à augmenter sans atteindre un niveau aussi bas qu’à l’epoch 8. De ce fait, ce CNN est en état de surapprentissage suite à l’epoch 8. Ceci explique pourquoi le nombre d’epochs optimal est 8. D’ailleurs, lors de l’entrainement des différentes configurations du CNN, la technique de early stopping à été utilisée. Suite à cette technique, la majorité des configuration ont été stoppées dans les epochs avoisinant l’epoch 8. L’utilisation de cette technique est discuté plus en détails à la question 5.
</div>
<br>

<div style="text-align: justify; text-decoration: underline;">Figure 4.cnn.1. Courbe du loss lors de la validation du CNN au cours de 100 epochs</div>

![vaLl_loss du baseline sur 100 epochs](logs/cnn/cnn_baseline_val_loss_100epochs.png)

# Question 5

### MLP

Configuration
<ul>
	<li>Tensorflow</li>
	<li> intel i7 6700</li>
		<ul>
			<il>4 Cores</il>
			<li>8 threads</il>
			<li>3.4 Ghz</il>
			<li>boost ~3.8 Ghz</il>
		</ul>
	</il>
<il>16gb DDR4 2133 Mhz </il>
</ul>

Nombre de couches : Le nombre de couches a une incidence directe sur l’accuracy et la précision du modèle en considérant un nombre d’epoch égal. Ceci pour l’accuracy des données d'entraînement ainsi que celui des données de validation. En effet, ceux-ci sont plus haut plus que le nombre de couches est élevé.

Nombre de perceptrons par couche : Un plus grand nombre de perceptrons permet d'améliorer légèrement l'accuracy ainsi que la précision au prix d’un loss un peu plus élevé sur les données de validation. D’un autre côté, un nombre de perceptrons réduit descend très largement l’accuracy du modèle et réduit légèrement la précision de celui-ci.

taux d’apprentissage : Un taux d’apprentissage plus bas ralentie clairement la vitesse d’augmentation de l’accuracy a chaque epoch tandis qu’un plus haut l’accélère légèrement. En échange un taux d’apprentissage réduit semble réduire les variations entre les epochs en général (très visible sur ceux du loss). De plus un taux d’apprentissage plus bas semble aussi réduire le loss du modèle tandis que le val loss semble être identique peut importe le taux d’apprentissage essayer.

Les différents paramètres ont des effets différents dépendamment de l’effet rechercher. Par exemple un nombre de couche ou de perceptron insuffisants peut empêcher d’atteindre un accuracy aussi bonne. 
Les autres paramètres peuvent aussi avoir des effets sur l’accuracy mais ce n’est pas nécessairement le but premier du paramètre. Par exemple, le learning rate a certe une influence sur l’accuracy, mais il est surtout intéressant de voir son influence sur la variabilité des résultats entre les epochs ce qui permet de pousser vers un modèle qui apprend plus vite mais qui varie beaucoup d’un epoch à l’autre ou un modèle qui varie peu mais apprend plus lentement. Finalement le nombre d’epoch n’a pas un impacte à proprement dite sur le processus d’apprentissage. En effet, celui-ci sert à arrêter plus ou moins tôt notre apprentissage pour éviter une situation de sur apprentissage ou de sous apprentissage.



In [38]:
nn_results_path = os.path.join(constants.PROJECT_ROOT_PATH, constants.LOGS_PATH, 'NN', 'NNResult.json')

with open(nn_results_path, 'r') as file:
    nn_run_results = json.load(file, object_pairs_hook=OrderedDict)

dataset_size_run_results = OrderedDict()
dataset_size_run_results['0'] = nn_run_results['0']
dataset_size_run_results['1'] = nn_run_results['9']
dataset_size_run_results['2'] = nn_run_results['10']
dataset_size_run_results['3'] = nn_run_results['11']
del nn_run_results['9']
del nn_run_results['10']
del nn_run_results['11']

print('Tableau 5.mlp.1. Comparaison des différentes configurations du MLP')
display(pd.DataFrame(nn_run_results).reindex(['run_name', 'epoch', 'learning rate', 'layer: 0', 'layer: 1', 'layer: 2', 'layer: 3', 'layer: 4', 'accuracy', 'val_accuracy', 'f1', 'val_f1', 'train_time (seconds)']))

Tableau 5.mlp.1. Comparaison des différentes configurations du MLP


,0,1,2,3,4,5,6,7,8
run_name,NN/BaseLine,NN/CouchePlus,NN/CoucheMoins,NN/PerceptronPlus,NN/PerceptronMoins,NN/EpochPlus,NN/EpochMoins,NN/LearningRatePlus,NN/LearningRateMoins
epoch,60,60,60,60,60,200,20,60,60
learning rate,0.005,0.005,0.005,0.005,0.005,0.005,0.005,0.01,0.0025
layer: 0,100,100,100,500,20,100,100,100,100
layer: 1,100,100,2,500,20,100,100,100,100
layer: 2,2,100,None,2,2,2,2,2,2
layer: 3,None,100,None,None,None,None,None,None,None
layer: 4,None,2,None,None,None,None,None,None,None
accuracy,0.487579,0.778316,0.263308,0.577924,0.192407,0.786337,0.192555,0.522919,0.419451
val_accuracy,0.48906,0.763601,0.266115,0.513601,0.189385,0.792578,0.203282,0.506801,0.408634


<br>
<div style="text-align: justify; text-decoration: underline;">Figure 5.mlp.3. Courbes de l'accuracy de l'entrainement des 9 configurations du MLP</div>

![val_acc](logs/NN/nn_accuracy.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 5.mlp.2. Courbes de l'accuracy de la validation des 9 configurations du MLP</div>

![acc](logs/NN/nn_val_accuracy.png)

### CNN

Configuration matérielle utilisée pour les tests du CNN :
<ul>
	<li>tensorflow-gpu 1.13.1</li>
    <li>Nvidia GTX 960M
        <ul>
            <li>4 GB GDDR5</li>
            <li>1.2 GHz</li>
            <li>640 CUDA Cores</li>
        </ul>
    </li>
    <li>Intel i7-6700HQ
        <ul>
            <li>4 Cores</li>
            <li>8 Threads</li>
            <li>3.5 GHz</li>
        </ul>
    </li>
    <li>8 GB DDR4</li>    
</ul>

In [11]:
run_result_files = [
    'baseline_run_results.json',
    'higher_batch_size_run_results.json',
    'lower_batch_size_run_results.json',
    'more_conv_layers_run_results.json',
    'less_conv_layers_run_results.json',
    'higher_learning_rate_run_results.json',
    'lower_learning_rate6_run_results.json'
]

cnn_runs_results = []

for run_result_filename in run_result_files:
    with open(os.path.join(constants.PROJECT_ROOT_PATH, constants.CNN_LOGS_PATH, run_result_filename), 'r') as file:
        cnn_runs_results.append(json.load(file, object_pairs_hook=OrderedDict))
    
cnn_runs_results_pd = pd.DataFrame(cnn_runs_results).transpose()

print('Tableau 5.cnn.1. Comparaison des différentes configurations du CNN')
display(cnn_runs_results_pd)

Tableau 5.cnn.1. Comparaison des différentes configurations du CNN


,0,1,2,3,4,5,6
run_name,baseline,higher_batch_size,lower_batch_size,more_conv_layers,less_conv_layers,higher_learning_rate,lower_learning_rate6
config,None,{'batch_size': 150},{'batch_size': 10},{'layers': 'more'},{'layers': 'less'},{'learning_rate': 0.01},{'learning_rate': 0.0001}
train_time (seconds),199.516,199.463,168.888,212.201,214.762,96.3423,428.803
best_epoch,7,7,3,7,8,3,21
best_loss,0.0884091,0.146982,0.126399,0.0816567,0.0798499,0.693126,0.0719804
best_val_loss,0.0915121,0.141849,0.0891519,0.0713093,0.122723,0.692742,0.0918742
best_accuracy,0.968579,0.943368,0.956454,0.970279,0.97161,0.513529,0.97442
best_val_accuracy,0.968953,0.946186,0.969545,0.978119,0.964222,0.516263,0.970727
best_f1,0.968579,0.943368,0.956454,0.970279,0.97161,0.513529,0.97442
best_val_f1,0.968953,0.946186,0.969545,0.978119,0.964222,0.516263,0.970727


<br>
<div style="text-align: justify; text-decoration: underline;">Figure 5.cnn.3. Courbes de l'accuracy de l'entrainement des 7 configurations du CNN</div>

![val_acc](logs/cnn/cnn_acc.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 5.cnn.2. Courbes de l'accuracy de la validation des 7 configurations du CNN</div>

![acc](logs/cnn/cnn_val_acc.png)

![legend](logs/cnn/legend.png)

<div style="text-align: justify">
D’abord, c’est important de mentionner que la technique de early stopping a été utilisée pour tester les configurations du CNN présentées ci-dessus. Cette technique permet d’arrêter l'entraînement de l’algorithme lorsqu'une certaine condition est rencontrée. Cette condition est l’exécution d’un certain nombre d’epochs sans diminution du loss de la validation. Pour toutes les configurations présentées à l’exception de lower_learning_rate6, le nombre d’epoch limite est de 2. Pour la configuration lower_learning_rate6, le nombre d’epoch est de 3.
<br><br>
</div>
<div style="text-align: justify">
Concernant l’augmentation du batch_size ou la diminution de celui-ci par rapport à la configuration baseline, le tableau 5.cnn.1 montre que les performances du CNN sont définis par la relation 5.cnn.1 pour les configurations higher_batch_size, baseline et lower_batch_size.
</div>
<br>
$$higher\_batch\_size < baseline < lower\_batch\_size \enspace (5.cnn.1)$$
<br>
<div style="text-align: justify">
En effet, le loss de la validation correspond à la relation inverse soit la relation 5.cnn.2.
</div>
<br>
$$higher\_batch\_size > baseline > lower\_batch\_size \enspace (5.cnn.2)$$
<br>
<div style="text-align: justify">
Parallèlement, l’accuracy de la validation répond aussi à la relation 5.cnn.1.
<br><br>
Sans surprise, le temps d’exécution d’une epoch, définit par le rapport entre le temps d'entraînement et le nombre d’epochs, suit la relation 5.cnn.4.
</div>
<br>
$$higher\_batch\_size < baseline < lower\_batch\_size \enspace (5.cnn.3)$$
<br>
<div style="text-align: justify">
Cela s’explique par le fait qu’une batch_size d’une taille plus grande augmente le nombre d’observations utilisés pour l'entraînement à chaque epoch. Cela explique aussi pourquoi la performance de higher_batch_size est inférieur à celle des deux autres configurations. En termes simples, cette configuration tente d’apprendre en prenant de plus grosses bouchés et ce plus rapidement. Cela n’est pas étonnant que la performance de cette configuration soit moindre. 
<br><br>
Par la suite, pour la variation du nombre de couches de convolutions, le phénomène est inversé. Les performances des configurations more_conv_layers, baseline et less_conv_layers suivent la relation 5.cnn.4.
</div>
<br>
$$more\_conv\_layers > baseline > less\_conv\_layers \enspace (5.cnn.4)$$
<br>
<div style="text-align: justify">
En même temps, l’accuracy de la validation suit la relation 5.cnn.4. Inversement, le loss de la validation suit la relation inverse. Pour le temps d’exécution d’une epoch, la relation précédente s’applique aussi. Le niveau d’accuracy accrue et le loss inférieur liés à l’augmentation du nombre de couches de convolution s'expliquer par le fait que le CNN dispose de plus de degré de liberté pour apprendre les particularité des exemples d’entrainements. Cependant, le nombre supérieur de couche de convolution entraîne un temps d'entraînement supérieur.
<br><br>
En ce qui concerne le taux d’apprentissage, la performance des configurations higher_learning_rate, baseline et lower_learning_rate6 suit la relation 5.cnn.5.
</div>
<br>
$$higher\_learning\_rate < baseline < lower\_learning\_rate \enspace (5.cnn.5)$$
<br>
<div style="text-align: justify">
L’accuracy de la validation et le temps d’exécution d’une epoch suivent la relation 5.cnn.5. Pour sa part, le loss de la validation suit la relation inverse à 5.cnn.5. Le fait que le taux d’apprentissage diminut les performances s’explique en terme simple par le fait que l’algorithme essaie d’apprendre trop rapidement. En effet, lors de l’optimisation de la fonction d’erreur, l’algorithme saute par dessus le minimum global.
<br><br>
Enfin, notez que la structure des configurations less_conv_layers, baseline et more_conv_layers est présentée dans les figures 5.cnn.4 à 5.cnn.6. 
</div>

In [15]:
cnn_model_less_conv = models.load_model(os.path.join(constants.PROJECT_ROOT_PATH, constants.CNN_MODELS_PATH, 'less_conv_layers.h5'))

print('Figure 5.cnn.4. Détails des couches du CNN less_conv_layers')
cnn_model_less_conv.summary()

Figure 5.cnn.4. Détails des couches du CNN less_conv_layers
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_368 (Conv2D)          (None, 140, 140, 3)       84        
_________________________________________________________________
conv2d_369 (Conv2D)          (None, 138, 138, 3)       84        
_________________________________________________________________
max_pooling2d_184 (MaxPoolin (None, 69, 69, 3)         0         
_________________________________________________________________
conv2d_370 (Conv2D)          (None, 69, 69, 6)         168       
_________________________________________________________________
conv2d_371 (Conv2D)          (None, 67, 67, 6)         330       
_________________________________________________________________
max_pooling2d_185 (MaxPoolin (None, 33, 33, 6)         0         
_________________________________________________________________
conv2d_372 (Conv

In [17]:
cnn_model_baseline = models.load_model(os.path.join(constants.PROJECT_ROOT_PATH, constants.CNN_MODELS_PATH, 'baseline.h5'))

print('Figure 5.cnn.5. Détails des couches du CNN baseline')
cnn_model_baseline.summary()

Figure 5.cnn.5. Détails des couches du CNN baseline
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_344 (Conv2D)          (None, 140, 140, 3)       84        
_________________________________________________________________
conv2d_345 (Conv2D)          (None, 138, 138, 3)       84        
_________________________________________________________________
max_pooling2d_172 (MaxPoolin (None, 69, 69, 3)         0         
_________________________________________________________________
conv2d_346 (Conv2D)          (None, 69, 69, 6)         168       
_________________________________________________________________
conv2d_347 (Conv2D)          (None, 67, 67, 6)         330       
_________________________________________________________________
max_pooling2d_173 (MaxPoolin (None, 33, 33, 6)         0         
_________________________________________________________________
conv2d_348 (Conv2D)     

In [16]:
cnn_model_more_conv = models.load_model(os.path.join(constants.PROJECT_ROOT_PATH, constants.CNN_MODELS_PATH, 'more_conv_layers.h5'))

print('Figure 5.cnn.6. Détails des couches du CNN more_conv_layers')
cnn_model_more_conv.summary()

Figure 5.cnn.6. Détails des couches du CNN more_conv_layers
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_374 (Conv2D)          (None, 140, 140, 3)       84        
_________________________________________________________________
conv2d_375 (Conv2D)          (None, 138, 138, 3)       84        
_________________________________________________________________
max_pooling2d_187 (MaxPoolin (None, 69, 69, 3)         0         
_________________________________________________________________
conv2d_376 (Conv2D)          (None, 69, 69, 6)         168       
_________________________________________________________________
conv2d_377 (Conv2D)          (None, 67, 67, 6)         330       
_________________________________________________________________
max_pooling2d_188 (MaxPoolin (None, 33, 33, 6)         0         
_________________________________________________________________
conv2d_378 (Conv

# Question 6

#### Méthode utilisée afin de trouver le meilleur modèle SVM
<br>
<div style="text-align: justify">
Dans un premier temps, il est important de rappeler que la méthode StandardScaler ainsi que la fonction fit_transform de sklearn ont été utilisées pour normaliser et transformer le jeu de données du SVM. Plusieurs raisons expliquent l’utilisation de la méthode StandardScaler pour normaliser les données. D’ailleurs, l’une d’entre elles est pour l’amélioration de la rapidité d’exécution du SVM. À haut niveau, cette méthode permet de normaliser les données en supprimant la moyenne et en effectuant une mise à l’échelle à la variance d’unité. Par exemple, le SVM avec noyau RBF suppose que toutes les données sont centrées autour de 0 et présentent une variance dans le même ordre. Si une caractéristique a une variance supérieure de plusieurs ordres de grandeur, elle peut rendre l’estimateur incapable d’apprendre correctement. La fonction fit_transform de son côté adapte simplement le transformateur à la matrice (données galaxies) et renvoie une version transformée de la matrice en question.
<br><br>
En deuxième lieu, la fonction train_test_split de sklearn a été utilisée pour diviser le jeu de données en deux matrices pour l'entraînement et pour tester les sous-ensembles. La taille du jeu de données utilisé pour l'entraînement est de 80% et celui des tests est de 20%.
<br><br>
En troisième lieu, la méthode GridSearchCV de sklearn a été utilisée pour évaluer toutes les combinaisons possibles entre les hyperparamètres spécifiés dans ce laboratoire. Même si la recherche par grille coûte cher, l’objectif était d’identifier l’hyperparamètre qui maximise les résultats avec une recherche exhaustive sur un ensemble de données de validation représentant un sous-ensemble de données d'entraînement. Dans le cas présent du laboratoire, la méthode GridSearchCV de sklearn a été utilisée pour effectuer une recherche en grille avec SVM linéaire et SVM non-linéaire (RBF).
</div>

#### Résultats

<br>
<div style="text-align: justify">
Pour ce qui est des résultats obtenus avec la recherche par grille sur SVM linéaire, quatre combinaisons ont été testées à partir des quatre valeurs de C (1E-3, 1E-1, 1.0 et 10.0). Les meilleurs paramètres trouvés pour le SVM linéaire afin d’identifier la meilleure précision sont C = 10, class_weight = balanced, gamma = scale et kernel = linear. Ces paramètres ont permis d’atteindre une précision de 0.9529 sur un total de 24.7 secondes. Les résultats obtenus avec C = 10 sur la précision de l'entraînement, la précision des tests, la précision d'entraînement F1 et la précision des tests F1 tournent tous autour d’une précision de 0.95. Effectuer les tests sur les 4 combinaisons a pris un total de 59.49 secondes.
</div>

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 6.1. Résultat de la recherche en grille du SVM linéaire</div>

![results_svm_lineaire](logs/svm/results_svm_lineaire.png)

<br>
<div style="text-align: justify">
Pour ce qui est des résultats obtenus avec la recherche par grille sur SVM non linéaire (RBF), 16 combinaisons ont été testées à partir des quatre valeurs de C (1E-3, 1E-1, 1.0 et 10.0) et des quatre valeurs pour gamma (1E-3, 1E-1, 1.0 et 10.0). Les meilleurs paramètres trouvés pour le SVM non linéaire (RBF) afin d’identifier la meilleure précision sont C = 10, gamma = 1E-3 (0.001) et kernel = rbf. Ces paramètres ont permis d’atteindre une précision de 0.9562. Effectuer les tests sur les 16 combinaisons a pris un total de 5.4 minutes.
</div>
<br>
<div style="text-align: justify; text-decoration: underline;">Figure 6.2. Accuracy de la phase d'entrainement lors de la recherche en grille du SVM avec noyaut RBF</div>

![resultats_rbf_train_acc](logs/svm/resultats_rbf_train_acc.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 6.3. Accuracy de la phase de test lors de la recherche en grille du SVM avec noyaut RBF</div>

![resultats_rbf_test_acc](logs/svm/resultats_rbf_test_acc.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 6.4. Mesure F1 de la phase d'entrainement lors de la recherche en grille du SVM avec noyaut RBF</div>

![resultats_rbf_train_f1](logs/svm/resultats_rbf_train_f1.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 6.5. Mesure F1 de la phase de test lors de la recherche en grille du SVM avec noyaut RBF</div>

![resultats_rbf_test_f1](logs/svm/resultats_rbf_test_f1.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 6.6. Temps d'entrainement lors de la recherche en grille du SVM avec noyaut RBF</div>

![train_time](logs/svm/train_time.png)

#### Impacts des hyperparamètres et leur utilité respective

<br>
<div style="text-align: justify">
Dans un premier temps, le modèle SVM cherche à maximiser la marge entre les hyperplans formés par les vecteurs de support tout en ayant une classification idéale. Naturellement, maximiser la marge entre les hyperplans a comme effet de minimiser le coût. Cependant, le coût est minimisé seulement si chaque échantillon du vecteur test est classifié adéquatement.
<br><br>
En second lieu, il faut souligner que plusieurs cas sont possibles. En effet, on peut y retrouver le cas où les échantillons sont non séparables et le cas où les classes sont non-séparables linéairement. Pour les échantillons qui sont non-séparables, l’hyperparamètre C est utilisée pour équilibrer la marge et pour minimiser la fonction du coût. Plus la valeur de C est grande, plus la tolérance est basse et plus la marge diminue. Pour les classes sont non-séparables linéairement, les vecteurs d’entrées sont insérés dans un espace possédant une plus grande dimension. Ensuite, les vecteurs sont traités dans une transformation de type non linéaire grâce à la fonction du noyau RBF qui possède les fonctions nécessaires pour simplifier le calcul.
</div>

# Question 7

### SVM
<div style="text-align: justify">
L’impact de la taille de l’ensemble d’apprentissage sur les performances du SVM linéaire et non linéaire (RBF) est très évident. Pour SVM linéaire, les paramètres suivants ont été utilisés: C = 1.0, class_weight = balanced, gamma = scale et kernel = linear. Pour SVM non linéaire (RBF), les paramètres suivants ont été utilisés: C = 10, gamma = 1E-3 (0.001) et kernel = rbf.
<br><br>

Avec SMV linéaire, une taille de 25% donne un temps d’exécution de 0.425 seconde, une taille de 50% donne un temps d’exécution de 1.67 seconde et une taille de 100% donne un temps d’exécution de 7.66 secondes. Il est donc évident que plus la taille est grande, moins le modèle SVM linéaire est performant.
<br><br>

Avec SVM non linéaire (RBF), une taille de 25% donne un temps d’exécution de 0.287 seconde, une taille de 50% donne un temps d’exécution de 1.012 secondes et une taille de 100% donne un temps d’exécution de 4.1713 secondes. Une fois de plus, il est donc évident que plus la taille est grande, moins le modèle SVM non linéaire (RBF) est performant.
<br><br>

Les résultats de la précision du SVM linéaire et non linéaire (RBF) montrent une différence négligeable qui est de plus ou moins 1% entre une taille de 25% et une taille de 100%. Bref, même si le modèle SVM non linéaire (RBF) est plus performant que celui du SVM linéaire, le temps d’exécution en fonction de la taille augmente exponentiellement pour une faible amélioration de la précision.
</div>

<br>
<div style="text-align: justify; text-decoration: underline;">Tableau 7.svm.1. Comparaison du mod'ele SVm lin/aire et avec RBF avec 100%, 50% et 25% des données</div>

![dataset_size](logs/svm/dataset_size.png)

### MLP

La ligne bleue de la figure 7.mlp.1 ci-dessous représente le baseline avec toutes les valeurs et est sensiblement celle qui en général offre la meilleur accuracy sur les données de validation. Elle est suivi de très proche par le baseline avec 75% des données donc on peut en déduire que entre 100% ou 75% des données il y a peu d’impacte. 
En contrepartie, l’exécution avec 50% et 25% des données montre une accuracy sur les données de validation fortement réduite dans la figure 7.mlp.2. En effet on peut remarquer une descente de 8% d’accuracy entre le baseline avec 100% des données et l’exécution avec 25% des données.

In [39]:
print('Tableau 7.mlp.1. Tableau comparatif de la performance du MLP avec 100%, 75%, 50% et 25% des données')
display(pd.DataFrame(dataset_size_run_results).reindex(['run_name', 'epoch', 'learning rate', 'layer: 0', 'layer: 1', 'layer: 2', 'accuracy', 'val_accuracy', 'f1', 'val_f1', 'train_time (seconds)']))

Tableau 7.mlp.1. Tableau comparatif de la performance du MLP avec 100%, 75% 50% et 25% des données


,0,1,2,3
run_name,NN/BaseLine,NN/BaseLine75,NN/BaseLine50,NN/BaseLine25
epoch,60,60,60,60
learning rate,0.005,0.005,0.005,0.005
layer: 0,100,100,100,100
layer: 1,100,100,100,100
layer: 2,2,2,2,2
accuracy,0.487579,0.494381,0.448026,0.40491
val_accuracy,0.48906,0.476153,0.445594,0.407801
f1,0.993494,0.99517,0.996008,0.996155
val_f1,0.959196,0.955459,0.959196,0.949173


Légende:
- Orange : BaseLine (100% des données)
- Rouge : BaseLine75 (75% des données)
- Bleu : BaseLine50 (50% des données)
- Rose : BaseLine25 (25% des données)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.mlp.1. Courbe de l'accuracy de l'entraînement du MLP avec 100%, 75%, 50% et 25% des données</div>

![acc_50p25p](logs/NN/nn_acc_size_test.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.mlp.2. Courbe de l'accuracy de la validation du MLP avec 100%, 75%, 50% et 25% des données</div>

![acc_50p25p](logs/NN/nn_val_acc_size_test.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.mlp.3. Courbe de loss de l'entrainement du MLP avec 100%, 75%, 50% et 25% des données</div>

![acc_50p25p](logs/NN/nn_loss_size_test.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.mlp.4. Courbe de loss de la validation du MLP avec 100%, 75%, 50% et 25% des données</div>

![acc_50p25p](logs/NN/nn_val_loss_size_test.png)

### CNN
<br>
<div style="text-align: justify">
L’impact de la taille de l’ensemble des données sur les performances du CNN est très claire. Incontestablement, la taille de cet ensemble à de l’importance. Ainsi, plus cette taille est grande, plus les performances sont grandes. En fait, tel que présenté par les figures 7.cnn.1 et 7.cnn.2, l’accuracy sur l’ensemble d'entraînement et sur celui de validation diminue avec la réduction de la taille de l’ensemble de données. Parallèlement, les figures 7.cnn.3 et 7.cnn.4 montrent que le loss diminue lorsque la taille de l’ensemble de données augmente.
<br><br>
Néanmoins, les performances du CNN ne sont pas proportionnel à la taille de l’ensemble de données. Dans les faits, une réduction de la taille de cet ensemble de 50 % correspond seulement à une baisse de 1,74 % de l’accuracy sur l’ensemble de validation. 
</div>

In [18]:
run_result_files = [
    'baseline_run_results.json',
    'baseline_50percent2_run_results.json',
    'baseline_25percent_run_results.json'
]

cnn_runs_results = []

for run_result_filename in run_result_files:
    with open(os.path.join(constants.PROJECT_ROOT_PATH, constants.CNN_LOGS_PATH, run_result_filename), 'r') as file:
        cnn_runs_results.append(json.load(file, object_pairs_hook=OrderedDict))
    
cnn_runs_results_pd = pd.DataFrame(cnn_runs_results).transpose()
print('Tableau 7.cnn.1. Tableau comparatif de la performance du CNN avec 100%, 50% et 25% des données')
display(cnn_runs_results_pd)

Tableau 7.cnn.1. Tableau comparatif de la performance du CNN avec 100%, 50% et 25% des données


,0,1,2
run_name,baseline,baseline_50percent2,baseline_25percent
config,None,None,None
train_time (seconds),199.516,86.7826,68.0494
best_epoch,7,5,7
best_loss,0.0884091,0.118315,0.131413
best_val_loss,0.0915121,0.124777,0.193495
best_accuracy,0.968579,0.956232,0.949719
best_val_accuracy,0.968953,0.951508,0.92435
best_f1,0.968579,0.956232,0.949719
best_val_f1,0.968953,0.951508,0.92435


Légende:
- Orange : baseline (100% des données)
- Bleu : baseline_50percent2 (50% des données)
- Rouge : baseline_25percent (25% des données)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.cnn.1. Courbe de l'accuracy de l'entraînement du CNN avec 100%, 50% et 25% des données sur 8 epochs</div>

![acc_50p25p](logs/cnn/cnn_acc_50p25p.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.cnn.2. Courbe de l'accuracy de la validation du CNN avec 100%, 50% et 25% des données sur 8 epochs</div>

![val_acc_50p25p](logs/cnn/cnn_val_acc_50p25p.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.cnn.3. Courbe de loss de l'entraînement du CNN avec 100%, 50% et 25% des données sur 8 epochs</div>

![loss_50p25p](logs/cnn/cnn_loss_50p25p.png)

<br>
<div style="text-align: justify; text-decoration: underline;">Figure 7.cnn.4. Courbe de loss de la validation du CNN avec 100%, 50% et 25% des données sur 8 epochs</div>

![val_loss_50p25p](logs/cnn/cnn_val_loss_50p25p.png)


# Question 8
<br>
Le problème de classification des galaxies semble être mieux répondu avec l’algorithme des réseaux convolutifs (CNN). Les algorithmes de classification SVM et CNN ont chacun des avantages qui méritent d’être mis à la lumière.

Dans un premier temps, l’algorithme CNN fonctionne bien avec la reconnaissance d’images et fonctionne généralement mieux avec de très grands ensembles de données, ce qui ne sera pas nécessairement le cas pour SVM. Comme il a été vu, l’algorithme CNN augmente la complexité du modèle en ajoutant plus de couches, ce qui n’est pas possible avec SVM. En d’autres mots, il n’est pas possible de complexifier le modèle de l’algorithme des machines à vecteurs de support.

Le CNN supporte aisément une entrée de quelques milliers d’images de galaxies avec un hyperparamètre portant sur nombre d’itérations se rapprochant de 100 pour obtenir une précision pouvant dépasser les 96%. Pour atteindre un tel résultat, seulement entre 10 et 15 minutes d'entraînement seraient nécessaires. Aussi, le CNN gagne beaucoup en précision lorsque les données sont structurées de manière à être exploitables par l’architecture.

Dans un deuxième temps, malgré un penchant vers l’utilisation du CNN pour répondre au problème de classification des images de galaxies, il pourrait tout de même être dit du SVM qu’il est conceptuellement plus simple que CNN. Cette caractéristique facilite les analyses statistiques préliminaires aussi appelées post-hoc. Dans plusieurs applications en ingénierie, l’ingénieur peut être appelé à remplir des critères de qualité et de sécurité très spécifique. La complexité du CNN peut donc s’avérer être un problème majeur lorsqu’il serait temps d’analyser ce que le réseau fait réellement. 

Finalement, en comparant les résultats obtenus avec SVM et CNN, il n’est pas difficile de constater que le temps nécessaire au SVM pour entraîner ses données est plus long que celui du CNN. Pourtant, malgré un temps d’exécution qui est plus long, les deux algorithmes ont des résultats relativement similaires en terme de précision. Cependant, il faut tout de même souligner que CNN penche vers une meilleure précision de SVM.  


# Question 9
<br>
<div style="text-align: justify">
Concernant le CNN, une première amélioration est d’utiliser des images d’entrées qui ont une plus grande résolution. À vrai dire, ce rapport a démontré qu’une accuracy d’environ 98 % peut être atteinte à l’aide d’images de taille 140x140 pixels. Néanmoins, utiliser des images d’une plus grande taille, par exemple de taille originale soit 424x424 pixels, permettrait au CNN d’apprendre plus de primitives. Hypothétiquement, cela permettrait à cet algorithme d’atteindre des performances plus importantes. Parallèlement, la taille du réseau pourrait aussi être agrandie.
<br><br>
De ce point de vue, ce rapport a prouvé que les performances du CNN baseline augmentent après l’ajout de couches de convolution telle que présenté à la question 5. La question qui devrait plutôt être répondue est : quel est le nombre de couches additionnelles optimal?
<br><br>
Par ailleurs, pour le MLP et le CNN, un meilleur algorithme d’optimisation pourrait améliorer les performances de ces modèles. À l’instant, ces modèles présentés dans ce rapport utilisent l’algorithme d’optimisation RMSProp. Cet algorithme utilise une méthode pour dynamiquement adapter le taux d’apprentissage. Cependant, en 2015 Kingma et Ba ont présenté la méthode Adam qui ajoute entre autres un terme de correction du biais à la méthode RMSProp. Lors d’une expérimentation, ils ont démontré que la méthode Adam obtient des performances égales ou supérieures à celles de RMSProp (Kingma et Ba, 2015, p. 8). Ainsi, la méthode Adam pourrait potentiellement améliorer les performances du MLP et du CNN. Inversement, une alternative est d’utiliser une méthode avec un taux d’apprentissage non adaptatif tel que la descente de gradient stochastique et de trouver le taux d’apprentissage optimal par expérimentation.
</div>

# Conclusion
<br>
<div style="text-align: justify">
Dans ce troisième laboratoire, le principal mandat était d’analyser et utiliser de nouvelles approches à l’aide de deux algorithmes dans le but de résoudre le problème de classification des galaxies. Les deux algorithmes en question sont les machines à vecteurs de support (SVM) et les réseaux neuronaux. Pour évaluer l’impact sur la classification des galaxies, plusieurs hyperparamètres ont été utilisés tels que le nombre de couches total, le nombre de perceptrons dans la couche cachée, le nombre d’itérations, le taux d’apprentissage, la taille du lot, la variable C et sigma. Après avoir exécuté et affiché les résultats des différents algorithmes, plusieurs observations de haut niveau peuvent être faites.
<br><br>
L’algorithme des réseaux neuronaux a su démontrer une forte sensibilité avec l’hyperparamètre du taux d’apprentissage et nous rappel de l’importance d’effectuer une analyse approfondie sur celui-ci afin de valider la pertinence du modèle. De plus, il a été observé que l’hyperparamètre du nombre de couches doit être ajusté en fonction de la complexité de la classification. En effet, une paire de couches pourrait être dans le cas présent suffisante pour résoudre le problème de classification. De plus, l’hyperparamètre du nombre de perceptrons rappel que un nombre trop élevé peut augmenter considérablement le temps de calcul pour entraîner le model, mais qu’un nombre trop faible entraîne une très grande chute de l’accuracy du modèle. Une amélioration intéressante aurait été de raffiner le baseline utilisé. En effet, avec nos expérimentation sur les hyperparamètres on peut facilement apercevoir que les hyperparamètres utilisé pour le baseline sont loin d’être les plus optimaux.
<br><br>
L’algorithme des machines à vecteurs de support (SVM) a su démontrer une grande performance et précision lorsque la taille du jeu de données est moindre. Il a aussi été observé que l’algorithme en question démontre une forte sensibilité aux valeurs aberrantes, ce qui vient supporter l’importance d’effectuer une mise à l’échelle des données d’apprentissage. Les résultats obtenus du SVM montrent que les différentes valeurs de l’hyperparamètre C ont directement un impact sur la tolérance des erreurs du classificateur. Après avoir analysé les différents résultats obtenus sur la précision, la meilleure valeur de l’hyperparamètre C est 10. Ce résultat confirme une forte intolérance sur les erreurs de données d’entrainements. D’ailleurs, l’intolérance en question assure une bonne généralisation sur les données de tests. Bref, le SVM linéaire est plus efficace avec des données qui sont linéairement distinguables tandis que le modèle SVM non linéaire (RBF) ne le sera pas, ce qui fait du RBF un meilleur modèle pour la classification des galaxies.
<br><br>
Il a été discuté qu’aucune méthode de mise à l’échelle ne peut être utilisée de façon universelle sur les différents algorithmes d’apprentissages. Puisque les classificateurs fonctionnent tous différemment, il est normal que la méthode à utiliser pour maximiser la précision soit différente d’un algorithme à un autre. Cependant, il n’a pas été discuté d’utiliser une série de méthodes de normalisation des données pour aller chercher les avantages de chacun et maximiser l’efficacité du processus de mise à l’échelle pour l’atteinte d’une plus grande précision.
</div>

# Bibliographie

Kingma, D.P., & Ba, J. (2015). Adam: A Method for Stochastic Optimization. CoRR, abs/1412.6980.

Michael A. Nielsen, "Neural Networks and Deep Learning", Determination Press, 2015